In [2]:
import json
import requests
import base64
import tensorflow as tf

def prepare_example(message):
  feature = {
      "text": tf.train.Feature(bytes_list=tf.train.BytesList(value=[message.encode()])),
  }

  example = tf.train.Example(features=tf.train.Features(feature=feature))

  example_str = example.SerializeToString()

  encoded_example = base64.b64encode(example_str).decode('utf-8')

  # Create the JSON data with the encoded example
  json_data = json.dumps({"signature_name": "serving_default", "instances": [encoded_example]})

  return json_data


def predict_with_model_server(message):
  json_data = prepare_example(message)
  print(json_data)

  endpoint = 'http://localhost:8080/v1/models/spam-classification-model:predict'

  response = requests.post(endpoint, data=json_data)

  prediction = response.json().get("predictions")

  if prediction:
    prediction_value = prediction[0][0]
    return f"Prediction: {prediction_value}"
  else:
    return f"Error: {response.text}"

message = "Dear Valued Customer, Congratulations! You've been selected for an exclusive opportunity to receive a special gift, absolutely FREE! But hurry, this offer is only available for a limited time! Claim your free gift now by clicking the link below and entering your details, Claim Your Free Gift Now. Don't miss out on this amazing opportunity to get something for nothing! Act fast before its too late! Warm Regards"

prediction = predict_with_model_server(message)
print(prediction)


TypeError: Object of type bytes is not JSON serializable

In [16]:
import tensorflow as tf

model = tf.saved_model.load("C:/Users/kicot/Tugas/colab/c-spam-pipeline/serving_model_dir/spam-classification-model/1715276941")

print(model.signatures["serving_default"].inputs)

[<tf.Tensor 'examples:0' shape=(None,) dtype=string>, <tf.Tensor 'unknown:0' shape=() dtype=resource>, <tf.Tensor 'unknown_0:0' shape=() dtype=int64>, <tf.Tensor 'unknown_1:0' shape=() dtype=string>, <tf.Tensor 'unknown_2:0' shape=() dtype=int64>, <tf.Tensor 'unknown_3:0' shape=() dtype=resource>, <tf.Tensor 'unknown_4:0' shape=() dtype=resource>, <tf.Tensor 'unknown_5:0' shape=() dtype=resource>, <tf.Tensor 'unknown_6:0' shape=() dtype=resource>, <tf.Tensor 'unknown_7:0' shape=() dtype=resource>, <tf.Tensor 'unknown_8:0' shape=() dtype=resource>, <tf.Tensor 'unknown_9:0' shape=() dtype=resource>, <tf.Tensor 'unknown_10:0' shape=() dtype=resource>, <tf.Tensor 'unknown_11:0' shape=() dtype=resource>, <tf.Tensor 'unknown_12:0' shape=() dtype=resource>]


## Docker Windows Powershell Script 

docker run -t --rm -d -p 8501:8501 -v "C:/Users/kicot/Tugas/colab/c-spam-pipeline/serving_model_dir/spam-classification-model/1715276941:/models/spam-classification-model/1" -e MODEL_NAME=spam-classification-model tensorflow/serving